# Data Engineering in Python with databolt - Find Token Clusters for Fuzzy Merging Identifiers (d6tlib/d6tjoin.utils)

## Introduction

Identifiers such as securities IDs often come in different conventions which makes joining them difficult. Normal joins don't work and fuzzy joins often get tripped up by commonly occuring tokens. 

In this notebook we will show how to use `d6tstack.utils.tokenCount` to find clusters of tokens and match on tokens.

In [15]:
import d6tjoin.utils
import d6tjoin.top1
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
import numpy as np

In [16]:
# data is tickers from two different vendors which try to join
df1 = pd.DataFrame({'id':["AAP","AAPL","APRN","AMZN-AMZN","BBW","NMG","JLP"]})
df2 = pd.DataFrame({'id':["AAP_US_Equity","AAPL_US_Equity","AMZN_US_Equity","APRN_US_Equity","AD_NA_Equity","BBY_US_Equity","BMW_NA_Equity","PRIVATE_NMG","PRIVATE_JLP"]})


In [17]:
# d6tjoin.utils.PreJoin() shows none of the ids match

d6tjoin.utils.PreJoin([df1,df2],['id']).stats_prejoin()

  key left key right  all matched  inner  left  right  outer  unmatched total  unmatched left  unmatched right
0       id        id        False      0     7      9     16               16               7                9
1  __all__   __all__        False      0     7      9     16               16               7                9


In [18]:
# attempt to join manually, better but still missing a few

df1['id_cleaned'] = df1['id'].str.split('-').str[0]
df2['id_cleaned'] = df2['id'].str.split('_').str[0]

d6tjoin.utils.PreJoin([df1,df2],['id_cleaned']).stats_prejoin()

     key left   key right  all matched  inner  left  right  outer  unmatched total  unmatched left  unmatched right
0  id_cleaned  id_cleaned        False      4     7      8     11                7               3                4
1     __all__     __all__        False      4     7      8     11                7               3                4


## Fuzzy joins get confused by tokens

Fuzzy joins to the rescue? Unfortunately, the presence of commonly occuring string tokens is messing with the string similarity functions.

In [19]:
# attempt a fuzzy join using edit distance => not looking good
d6tjoin.top1.MergeTop1(df1,df2,fuzzy_left_on=['id'],fuzzy_right_on=['id']).merge()['top1']['id']

D:\dev\d6tlib\d6tjoin\d6tjoin\top1.py:139: UserWarning: Top1 join for id has duplicates
  warnings.warn('Top1 join for %s has duplicates' %self.cfg_fuzzy_left_on)


,__top1left__,__top1right__,__matchtype__,__top1diff__
49,AAP,PRIVATE_JLP,top1 left,9
58,AAPL,PRIVATE_JLP,top1 left,9
9,AMZN-AMZN,AD_NA_Equity,top1 left,10
12,AMZN-AMZN,PRIVATE_NMG,top1 left,10
15,AMZN-AMZN,AMZN_US_Equity,top1 left,10
39,APRN,PRIVATE_NMG,top1 left,9
28,BBW,BBY_US_Equity,top1 left,11
30,BBW,PRIVATE_NMG,top1 left,11
31,BBW,PRIVATE_JLP,top1 left,11
32,BBW,BMW_NA_Equity,top1 left,11


In [20]:
# attempt a fuzzy join using affine gap distance => not looking good
import affinegap
d6tjoin.top1.MergeTop1(df1,df2,fuzzy_left_on=['id'],fuzzy_right_on=['id'], fun_diff=[affinegap.affineGapDistance]).merge()['top1']['id']

D:\dev\d6tlib\d6tjoin\d6tjoin\top1.py:139: UserWarning: Top1 join for id has duplicates
  warnings.warn('Top1 join for %s has duplicates' %self.cfg_fuzzy_left_on)


,__top1left__,__top1right__,__matchtype__,__top1diff__
52,AAP,AAP_US_Equity,top1 left,13.000
62,AAPL,AAPL_US_Equity,top1 left,14.000
15,AMZN-AMZN,AMZN_US_Equity,top1 left,64.625
38,APRN,APRN_US_Equity,top1 left,14.000
28,BBW,BBY_US_Equity,top1 left,23.000
32,BBW,BMW_NA_Equity,top1 left,23.000
7,JLP,AAP_US_Equity,top1 left,33.000
23,NMG,BMW_NA_Equity,top1 left,33.000


## Token-based clustering

With `d6tjoin.utils.splitcharTokenCount` you can quickly split the ids into tokens to find commonly occuring substrings. You can then use that knowledge to join the data.

In [21]:
dftoken=d6tjoin.utils.splitcharTokenCount(df2['id'])
print('*** token counts ***')
print(dftoken)
print('\n *** token occurance ***')
print(d6tjoin.utils.unique_contains(df2['id'], dftoken['word'].values))


*** token counts ***
      word  count
0   Equity      7
1       US      5
2       NA      2
3  PRIVATE      2

 *** token occurance ***
[('Equity', ['AAPL_US_Equity', 'AAP_US_Equity', 'AD_NA_Equity', 'AMZN_US_Equity', 'APRN_US_Equity', 'BBY_US_Equity', 'BMW_NA_Equity']), ('US', ['AAPL_US_Equity', 'AAP_US_Equity', 'AMZN_US_Equity', 'APRN_US_Equity', 'BBY_US_Equity']), ('NA', ['AD_NA_Equity', 'BMW_NA_Equity']), ('PRIVATE', ['PRIVATE_JLP', 'PRIVATE_NMG'])]


## Token-based joins

Based on the analysis above, we want to join pairs which have at least 1 common token. It's easy to define a function which computes that and pass that to `d6tjoin.top1.MergeTop1()` to get a good join.

In [22]:
import re
splitchars="[^a-zA-Z0-9]+"
def tokenmatch(s1,s2):
    s1=set(re.split(splitchars,s1))
    s2=set(re.split(splitchars,s2))
    return 3-len(s1 & s2)

d6tjoin.top1.MergeTop1(df1,df2,fuzzy_left_on=['id'],fuzzy_right_on=['id'], fun_diff=[tokenmatch], top_limit=[2]).merge()['top1']['id']


,__top1left__,__top1right__,__matchtype__,__top1diff__
52,AAP,AAP_US_Equity,top1 left,2
62,AAPL,AAPL_US_Equity,top1 left,2
15,AMZN-AMZN,AMZN_US_Equity,top1 left,2
38,APRN,APRN_US_Equity,top1 left,2
4,JLP,PRIVATE_JLP,top1 left,2
21,NMG,PRIVATE_NMG,top1 left,2


In [23]:
# note that we applied top_limit=[2], meaning strings should have at most 2 tokens mismatched, to exclude bad matches for BBW
d6tjoin.top1.MergeTop1(df1,df2,fuzzy_left_on=['id'],fuzzy_right_on=['id'], fun_diff=[tokenmatch]).merge()['top1']['id']


D:\dev\d6tlib\d6tjoin\d6tjoin\top1.py:139: UserWarning: Top1 join for id has duplicates
  warnings.warn('Top1 join for %s has duplicates' %self.cfg_fuzzy_left_on)


,__top1left__,__top1right__,__matchtype__,__top1diff__
52,AAP,AAP_US_Equity,top1 left,2
62,AAPL,AAPL_US_Equity,top1 left,2
15,AMZN-AMZN,AMZN_US_Equity,top1 left,2
38,APRN,APRN_US_Equity,top1 left,2
27,BBW,AD_NA_Equity,top1 left,3
28,BBW,BBY_US_Equity,top1 left,3
29,BBW,APRN_US_Equity,top1 left,3
30,BBW,PRIVATE_NMG,top1 left,3
31,BBW,PRIVATE_JLP,top1 left,3
32,BBW,BMW_NA_Equity,top1 left,3
